In [57]:
import numpy as np
import pandas as pd
import itertools

In [58]:
# Create a 100x100 array at elevation 0
numParticlesGridLength = 10
numAttr = 5
waterMolecules = np.zeros((numParticlesGridLength**2, numAttr), dtype=int)

df = pd.DataFrame(waterMolecules, columns=['x', 'y', 'elevation', 'temp', 'state'])
print(df)

    x  y  elevation  temp  state
0   0  0          0     0      0
1   0  0          0     0      0
2   0  0          0     0      0
3   0  0          0     0      0
4   0  0          0     0      0
.. .. ..        ...   ...    ...
95  0  0          0     0      0
96  0  0          0     0      0
97  0  0          0     0      0
98  0  0          0     0      0
99  0  0          0     0      0

[100 rows x 5 columns]


In [59]:
df['state'] = 'liquid'
print(df)

    x  y  elevation  temp   state
0   0  0          0     0  liquid
1   0  0          0     0  liquid
2   0  0          0     0  liquid
3   0  0          0     0  liquid
4   0  0          0     0  liquid
.. .. ..        ...   ...     ...
95  0  0          0     0  liquid
96  0  0          0     0  liquid
97  0  0          0     0  liquid
98  0  0          0     0  liquid
99  0  0          0     0  liquid

[100 rows x 5 columns]


In [60]:
combinations = list(itertools.product(range(numParticlesGridLength), repeat=2))
df['x'], df['y'] = zip(*combinations)

print(df)

    x  y  elevation  temp   state
0   0  0          0     0  liquid
1   0  1          0     0  liquid
2   0  2          0     0  liquid
3   0  3          0     0  liquid
4   0  4          0     0  liquid
.. .. ..        ...   ...     ...
95  9  5          0     0  liquid
96  9  6          0     0  liquid
97  9  7          0     0  liquid
98  9  8          0     0  liquid
99  9  9          0     0  liquid

[100 rows x 5 columns]


In [61]:
def warmUpTemp(df):
    random_increments = np.random.randint(0, 5, size=len(df))
    df['temp'] += random_increments
    return df


    x  y  elevation  temp   state
0   0  0          0     2  liquid
1   0  1          0     2  liquid
2   0  2          0     4  liquid
3   0  3          0     1  liquid
4   0  4          0     4  liquid
.. .. ..        ...   ...     ...
95  9  5          0     0  liquid
96  9  6          0     1  liquid
97  9  7          0     4  liquid
98  9  8          0     1  liquid
99  9  9          0     3  liquid

[100 rows x 5 columns]


In [66]:
for i in range(10):
    df = warmUpTemp(df)

df.loc[df['temp'] >= 100, 'state']= 'gas'
print(df)


    x  y  elevation  temp   state
0   0  0          0   106     gas
1   0  1          0   102     gas
2   0  2          0   104     gas
3   0  3          0   112     gas
4   0  4          0    79  liquid
.. .. ..        ...   ...     ...
95  9  5          0    98  liquid
96  9  6          0   101     gas
97  9  7          0   113     gas
98  9  8          0    93  liquid
99  9  9          0   112     gas

[100 rows x 5 columns]
